<h1, align=center> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;数据科学引论 - Python之道 </h1>

<h1, align=center> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;第4课 数据清洗 </h1>


<hr>

<h1, align="center">按国家和年份统计的CO2排放量</h1>

二氧化碳排放源自燃烧化石燃料和水泥制造，包含了在固体、液体和气体燃料消耗以及放空燃烧过程中所产生的二氧化碳

http://data.worldbank.org/indicator/EN.ATM.CO2E.PC/

<h2, align=center>获取数据</h2>

这些数据可以从世界银行(World Bank) [链接](http://data.worldbank.org/indicator/EN.ATM.CO2E.PC/) 或者从 Box [链接](https://ibm.box.com/shared/static/3yzxbbizo49bkl8cnjw15tymzfwkycj4.csv) 上下载

#### 我们在linux下可以使用bash命令 `wget` 从链接处获取 csv 文件

In [ ]:
!wget --output-document /resources/data/co2emissions.csv https://ibm.box.com/shared/static/3yzxbbizo49bkl8cnjw15tymzfwkycj4.csv

<hr>

<h2, align=center>使用 Pandas 导入数据</h2>

#### 导入所需的 `pandas` 库

In [1]:
import pandas as pd

#### 使用 `pd.read_csv` 导入数据

In [2]:
data = pd.read_csv("resources/data/co2emissions.csv", skiprows = 4)

#### 使用 `head` 显示 `data`  的前5行

In [4]:
data

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,Unnamed: 61
0,Aruba,ABW,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,NaN,NaN,NaN,NaN,NaN,NaN,...,26.230847,25.915833,24.670529,24.505835,13.155542,8.351294,8.408363,NaN,NaN,NaN
1,Afghanistan,AFG,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,0.046060,0.053604,0.073765,0.074233,0.086292,0.101467,...,0.154101,0.241723,0.293837,0.412017,0.350371,0.315602,0.299445,NaN,NaN,NaN
2,Angola,AGO,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,0.097472,0.079038,0.201289,0.192535,0.201003,0.191528,...,1.181527,1.232495,1.243406,1.252789,1.330843,1.254617,1.291328,NaN,NaN,NaN
3,Albania,ALB,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,1.258195,1.374186,1.439956,1.181681,1.111742,1.166099,...,1.484311,1.495600,1.578574,1.803715,1.692908,1.749211,1.978763,NaN,NaN,NaN
4,Andorra,AND,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,NaN,NaN,NaN,NaN,NaN,NaN,...,6.427887,6.121652,6.122595,5.867130,5.916597,5.900753,5.832170,NaN,NaN,NaN
5,Arab World,ARB,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,0.643689,0.685151,0.760855,0.874941,0.999098,1.165705,...,4.390401,4.542151,4.615758,4.537755,4.813631,4.650474,4.860234,NaN,NaN,NaN
6,United Arab Emirates,ARE,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,0.118758,0.108837,0.163018,0.175223,0.132355,0.146327,...,22.823947,21.908556,19.443690,19.076489,19.817684,18.954164,23.302023,NaN,NaN,NaN
7,Argentina,ARG,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,2.367473,2.442616,2.522392,2.316356,2.538379,2.641714,...,4.682912,4.410890,4.558500,4.600291,4.569384,4.462904,4.746797,NaN,NaN,NaN
8,Armenia,ARM,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,NaN,NaN,NaN,NaN,NaN,NaN,...,1.911538,1.509412,1.465622,1.710071,1.976060,1.899712,1.902759,NaN,NaN,NaN
9,American Samoa,ASM,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<h1, align=center>数据清洗</h1>

#### 观察这些数据，其质量有什么问题？我们应该如何解决这些问题？

例如，下面这些行有什么问题？

In [5]:
data.loc[[9,190,220,257]]

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,Unnamed: 61
9,American Samoa,ASM,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
190,Puerto Rico,PRI,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
220,Slovenia,SVN,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,NaN,NaN,NaN,NaN,NaN,NaN,...,8.586441,7.491602,7.485854,7.350638,7.185481,6.869551,6.213687,NaN,NaN,NaN
257,World,WLD,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,3.096937,3.067983,3.139054,3.243475,3.359831,3.438679,...,4.757954,4.657646,4.829611,4.969097,4.997730,4.989320,4.971572,NaN,NaN,NaN


## 数据质量的问题:
1. 有些行是多个国家的总和，而不是实际的国家(例如, "World").
2. 有些列是不相关的，可以被移除(例如, "Indicator Name").
3. 有些年份对于任何国家都没有数据(例如, 2015 到 2016).
4. 有些国家在任何年份都没有数据(例如, "American Samoa").

<br>

<h2> 1. 有些行是多个国家的总和，而不是实际的国家 (例如, "World"). </h2>

**目标:**  
移除不包含实际国家的行。幸运的是，世界银行(World Bank)提供了相应的元数据，表明了哪些行是国家，而哪些行是多个国家的总和。
- 导入countries_metadata.csv
- 在`Country Code`上将元数据与`data`合并

#### 获取 `countries_metadata.csv`

In [ ]:
!wget --output-document /resources/data/countries_metadata.csv https://ibm.box.com/shared/static/qh3o86mpij17ot7anydcmbwt41lwxvln.csv

#### 导入 `countries_metadata.csv`

In [6]:
metadata = pd.read_csv("resources/data/countries_metadata.csv", encoding = "utf-8")

In [7]:
metadata.head(10)

,Country Code,Region,IncomeGroup,SpecialNotes,TableName,Unnamed: 5
0,ABW,Latin America & Caribbean,High income,SNA data for 2000-2011 are updated from offici...,Aruba,NaN
1,AFG,South Asia,Low income,Fiscal year end: March 20; reporting period fo...,Afghanistan,NaN
2,AGO,Sub-Saharan Africa,Lower middle income,NaN,Angola,NaN
3,ALB,Europe & Central Asia,Upper middle income,NaN,Albania,NaN
4,AND,Europe & Central Asia,High income,WB-3 code changed from ADO to AND to align wit...,Andorra,NaN
5,ARB,NaN,NaN,Arab World aggregate. Arab World is composed o...,Arab World,NaN
6,ARE,Middle East & North Africa,High income,NaN,United Arab Emirates,NaN
7,ARG,Latin America & Caribbean,Upper middle income,National Institute of Statistics and Census re...,Argentina,NaN
8,ARM,Europe & Central Asia,Lower middle income,NaN,Armenia,NaN
9,ASM,East Asia & Pacific,Upper middle income,NaN,American Samoa,NaN


#### 如何标识列出的"Country Name" 是一个国家还是一个多国构成的区域?

注意，当某一行是像"Arab World"这样的聚合区域时,  `Region` 和 `IncomeGroup` 总是 NaN (Not a Number). 我们可以用这条规则来移除所有不是国家的区域.

#### 在关键字`Country Code`上合并`data` 与 `metadata`  

In [8]:
merge = pd.merge(data, metadata, on = "Country Code")

In [9]:
merge.head(10)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,Unnamed: 61,Region,IncomeGroup,SpecialNotes,TableName,Unnamed: 5
0,Aruba,ABW,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,NaN,NaN,NaN,NaN,NaN,NaN,...,8.351294,8.408363,NaN,NaN,NaN,Latin America & Caribbean,High income,SNA data for 2000-2011 are updated from offici...,Aruba,NaN
1,Afghanistan,AFG,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,0.046060,0.053604,0.073765,0.074233,0.086292,0.101467,...,0.315602,0.299445,NaN,NaN,NaN,South Asia,Low income,Fiscal year end: March 20; reporting period fo...,Afghanistan,NaN
2,Angola,AGO,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,0.097472,0.079038,0.201289,0.192535,0.201003,0.191528,...,1.254617,1.291328,NaN,NaN,NaN,Sub-Saharan Africa,Lower middle income,NaN,Angola,NaN
3,Albania,ALB,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,1.258195,1.374186,1.439956,1.181681,1.111742,1.166099,...,1.749211,1.978763,NaN,NaN,NaN,Europe & Central Asia,Upper middle income,NaN,Albania,NaN
4,Andorra,AND,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,NaN,NaN,NaN,NaN,NaN,NaN,...,5.900753,5.832170,NaN,NaN,NaN,Europe & Central Asia,High income,WB-3 code changed from ADO to AND to align wit...,Andorra,NaN
5,Arab World,ARB,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,0.643689,0.685151,0.760855,0.874941,0.999098,1.165705,...,4.650474,4.860234,NaN,NaN,NaN,NaN,NaN,Arab World aggregate. Arab World is composed o...,Arab World,NaN
6,United Arab Emirates,ARE,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,0.118758,0.108837,0.163018,0.175223,0.132355,0.146327,...,18.954164,23.302023,NaN,NaN,NaN,Middle East & North Africa,High income,NaN,United Arab Emirates,NaN
7,Argentina,ARG,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,2.367473,2.442616,2.522392,2.316356,2.538379,2.641714,...,4.462904,4.746797,NaN,NaN,NaN,Latin America & Caribbean,Upper middle income,National Institute of Statistics and Census re...,Argentina,NaN
8,Armenia,ARM,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,NaN,NaN,NaN,NaN,NaN,NaN,...,1.899712,1.902759,NaN,NaN,NaN,Europe & Central Asia,Lower middle income,NaN,Armenia,NaN
9,American Samoa,ASM,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,East Asia & Pacific,Upper middle income,NaN,American Samoa,NaN


**注意:** 当某一行不是实际的国家时，'Region'的值就是 NaN.

#### 移除 `Region` 为 NaN 的行

In [11]:
merge = merge[pd.notnull(merge['Region'])]

In [12]:
merge.head(10)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,Unnamed: 61,Region,IncomeGroup,SpecialNotes,TableName,Unnamed: 5
0,Aruba,ABW,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,NaN,NaN,NaN,NaN,NaN,NaN,...,8.351294,8.408363,NaN,NaN,NaN,Latin America & Caribbean,High income,SNA data for 2000-2011 are updated from offici...,Aruba,NaN
1,Afghanistan,AFG,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,0.046060,0.053604,0.073765,0.074233,0.086292,0.101467,...,0.315602,0.299445,NaN,NaN,NaN,South Asia,Low income,Fiscal year end: March 20; reporting period fo...,Afghanistan,NaN
2,Angola,AGO,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,0.097472,0.079038,0.201289,0.192535,0.201003,0.191528,...,1.254617,1.291328,NaN,NaN,NaN,Sub-Saharan Africa,Lower middle income,NaN,Angola,NaN
3,Albania,ALB,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,1.258195,1.374186,1.439956,1.181681,1.111742,1.166099,...,1.749211,1.978763,NaN,NaN,NaN,Europe & Central Asia,Upper middle income,NaN,Albania,NaN
4,Andorra,AND,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,NaN,NaN,NaN,NaN,NaN,NaN,...,5.900753,5.832170,NaN,NaN,NaN,Europe & Central Asia,High income,WB-3 code changed from ADO to AND to align wit...,Andorra,NaN
6,United Arab Emirates,ARE,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,0.118758,0.108837,0.163018,0.175223,0.132355,0.146327,...,18.954164,23.302023,NaN,NaN,NaN,Middle East & North Africa,High income,NaN,United Arab Emirates,NaN
7,Argentina,ARG,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,2.367473,2.442616,2.522392,2.316356,2.538379,2.641714,...,4.462904,4.746797,NaN,NaN,NaN,Latin America & Caribbean,Upper middle income,National Institute of Statistics and Census re...,Argentina,NaN
8,Armenia,ARM,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,NaN,NaN,NaN,NaN,NaN,NaN,...,1.899712,1.902759,NaN,NaN,NaN,Europe & Central Asia,Lower middle income,NaN,Armenia,NaN
9,American Samoa,ASM,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,East Asia & Pacific,Upper middle income,NaN,American Samoa,NaN
10,Antigua and Barbuda,ATG,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,0.662643,0.849084,1.796794,1.446821,1.540135,2.473585,...,5.360453,5.377649,NaN,NaN,NaN,Latin America & Caribbean,High income,NaN,Antigua and Barbuda,NaN


<br>

<h2>2. 有些列是不相关的，可以被移除.</h2>

**目标:**  
移除下列不相关的列:
- 第 3 列: **"Indicator Name"**
- 第 4 列: **"Indicator Code"**

In [13]:
merge.columns

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', 'Unnamed: 61', 'Region', 'IncomeGroup',
       'SpecialNotes', 'TableName', 'Unnamed: 5'],
      dtype='object')

In [15]:
? merge.drop

In [16]:
merge = merge.drop(merge.columns[[61,66]], axis=1) # Note: zero indexed
merge = merge.drop('Indicator Name', axis=1)
merge = merge.drop('Indicator Code', 1)

In [17]:
merge.columns

Index(['Country Name', 'Country Code', '1960', '1961', '1962', '1963', '1964',
       '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973',
       '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982',
       '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991',
       '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', 'Region',
       'IncomeGroup', 'SpecialNotes', 'TableName'],
      dtype='object')

<h2>3. 有些年份对于任何国家都没有数据.</h2>

**目标:**  
计算每一年的数据行数，NaN值不计算在内.


In [18]:
merge.count()

Country Name    217
Country Code    217
1960            151
1961            152
1962            154
1963            155
1964            161
1965            161
1966            161
1967            161
1968            161
1969            161
1970            163
1971            164
1972            166
1973            166
1974            166
1975            166
1976            166
1977            166
1978            166
1979            166
1980            166
1981            166
1982            166
1983            166
1984            166
1985            166
1986            167
1987            167
               ... 
1991            174
1992            195
1993            195
1994            196
1995            197
1996            197
1997            198
1998            199
1999            199
2000            199
2001            199
2002            200
2003            200
2004            200
2005            201
2006            202
2007            203
2008            203
2009            203


查看2015年，看起来所有行都不包含2015年的数据.

In [19]:
merge['2015']

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
6     NaN
7     NaN
8     NaN
9     NaN
10    NaN
11    NaN
12    NaN
13    NaN
14    NaN
15    NaN
16    NaN
17    NaN
18    NaN
19    NaN
20    NaN
21    NaN
22    NaN
23    NaN
24    NaN
25    NaN
26    NaN
27    NaN
28    NaN
29    NaN
30    NaN
       ..
225   NaN
226   NaN
229   NaN
230   NaN
231   NaN
232   NaN
234   NaN
236   NaN
239   NaN
240   NaN
241   NaN
242   NaN
243   NaN
244   NaN
245   NaN
247   NaN
248   NaN
249   NaN
250   NaN
251   NaN
252   NaN
253   NaN
254   NaN
255   NaN
257   NaN
258   NaN
259   NaN
260   NaN
261   NaN
262   NaN
Name: 2015, dtype: float64

#### 移除任何行都不包含数据的列

In [20]:
merge = merge.drop(['2015', '2016'], 1)

In [21]:
merge.count() #double-check that columns have been removed

Country Name    217
Country Code    217
1960            151
1961            152
1962            154
1963            155
1964            161
1965            161
1966            161
1967            161
1968            161
1969            161
1970            163
1971            164
1972            166
1973            166
1974            166
1975            166
1976            166
1977            166
1978            166
1979            166
1980            166
1981            166
1982            166
1983            166
1984            166
1985            166
1986            167
1987            167
               ... 
1989            167
1990            173
1991            174
1992            195
1993            195
1994            196
1995            197
1996            197
1997            198
1998            199
1999            199
2000            199
2001            199
2002            200
2003            200
2004            200
2005            201
2006            202
2007            203


<h2>4. 有些国家在任何年份都没有数据.</h2>

**目标:**  
使用行平均值来确定哪些国家不包含任何数据.

计算每一行的平均值(在 axis 1 上).

In [22]:
merge.mean(axis=1) #Takes the mean of all numeric quantities by row

0      20.486262
1       0.147484
2       0.600522
3       1.656215
4       6.888830
6      30.452819
7       3.633115
8       1.344459
9            NaN
10      4.824503
11     14.533135
12      7.198942
13      4.184062
14      0.030754
15     11.215788
16      0.209032
17      0.067407
18      0.194050
19      7.106923
20     20.284362
21     12.631206
22      3.971538
23      6.283099
24      1.345642
25      7.538987
26      0.945658
27      1.474204
28      3.231488
29     21.024915
30      0.390093
         ...    
225     4.328909
226     0.033163
229     0.212351
230     1.816982
231     0.453896
232     9.590787
234     0.220787
236     0.654379
239    16.614263
240     1.531793
241     2.423008
242     0.894346
243     0.119932
244     0.075384
245     7.206894
247     1.769533
248    19.134419
249     4.521001
250     0.938539
251     5.951815
252     3.803970
253          NaN
254     0.645479
255     0.515255
257     0.613257
258          NaN
259     0.719367
260     8.1327

正如你所见，在不包含任何数据的行中出现了 NaN.

#### 移除所有年份中不包含任何数据的行

In [23]:
merge = merge[pd.notnull(merge.mean(axis=1))]

<hr>

<h2, align = "center">数据清洗 ... 完成!</h2>

### 为了方便起见，重新命名

In [25]:
mydf = merge
mydf

,Country Name,Country Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2009,2010,2011,2012,2013,2014,Region,IncomeGroup,SpecialNotes,TableName
0,Aruba,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,25.915833,24.670529,24.505835,13.155542,8.351294,8.408363,Latin America & Caribbean,High income,SNA data for 2000-2011 are updated from offici...,Aruba
1,Afghanistan,AFG,0.046060,0.053604,0.073765,0.074233,0.086292,0.101467,0.107637,0.123734,...,0.241723,0.293837,0.412017,0.350371,0.315602,0.299445,South Asia,Low income,Fiscal year end: March 20; reporting period fo...,Afghanistan
2,Angola,AGO,0.097472,0.079038,0.201289,0.192535,0.201003,0.191528,0.246413,0.154912,...,1.232495,1.243406,1.252789,1.330843,1.254617,1.291328,Sub-Saharan Africa,Lower middle income,NaN,Angola
3,Albania,ALB,1.258195,1.374186,1.439956,1.181681,1.111742,1.166099,1.333055,1.363746,...,1.495600,1.578574,1.803715,1.692908,1.749211,1.978763,Europe & Central Asia,Upper middle income,NaN,Albania
4,Andorra,AND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.121652,6.122595,5.867130,5.916597,5.900753,5.832170,Europe & Central Asia,High income,WB-3 code changed from ADO to AND to align wit...,Andorra
6,United Arab Emirates,ARE,0.118758,0.108837,0.163018,0.175223,0.132355,0.146327,0.159950,5.383685,...,21.908556,19.443690,19.076489,19.817684,18.954164,23.302023,Middle East & North Africa,High income,NaN,United Arab Emirates
7,Argentina,ARG,2.367473,2.442616,2.522392,2.316356,2.538379,2.641714,2.792654,2.858162,...,4.410890,4.558500,4.600291,4.569384,4.462904,4.746797,Latin America & Caribbean,Upper middle income,National Institute of Statistics and Census re...,Argentina
8,Armenia,ARM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.509412,1.465622,1.710071,1.976060,1.899712,1.902759,Europe & Central Asia,Lower middle income,NaN,Armenia
10,Antigua and Barbuda,ATG,0.662643,0.849084,1.796794,1.446821,1.540135,2.473585,5.613448,8.912847,...,5.446757,5.539568,5.363407,5.418447,5.360453,5.377649,Latin America & Caribbean,High income,NaN,Antigua and Barbuda
11,Australia,AUS,8.582937,8.641569,8.835688,9.226440,9.759073,10.622321,10.328092,10.955625,...,18.200182,17.740845,17.538878,17.076819,16.103336,15.398600,East Asia & Pacific,High income,Fiscal year end: June 30; reporting period for...,Australia


<hr>

代码汇总:

In [26]:
import pandas as pd

#Download data
#!wget --output-document co2emissions.csv https://ibm.box.com/shared/static/3yzxbbizo49bkl8cnjw15tymzfwkycj4.csv
#!wget --output-document countries_metadata.csv https://ibm.box.com/shared/static/qh3o86mpij17ot7anydcmbwt41lwxvln.csv
    
#Import data
data = pd.read_csv("resources/data/co2emissions.csv", skiprows = 4)
metadata = pd.read_csv("resources/data/countries_metadata.csv", encoding = "utf-8")

#Merge data
merge = pd.merge(data, metadata, on = "Country Code")

#Remove non-country regions
merge = merge[pd.notnull(merge['Region'])]

#Drop some columns with no data
merge = merge.drop(merge.columns[[61,66]], axis=1)
merge = merge.drop(['Indicator Name', 'Indicator Code','2015', '2016'], 1)

#Drop some rows with no data
merge = merge[pd.notnull(merge.mean(axis=1))]

#Rename
mydf = merge

In [27]:
mydf

,Country Name,Country Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2009,2010,2011,2012,2013,2014,Region,IncomeGroup,SpecialNotes,TableName
0,Aruba,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,25.915833,24.670529,24.505835,13.155542,8.351294,8.408363,Latin America & Caribbean,High income,SNA data for 2000-2011 are updated from offici...,Aruba
1,Afghanistan,AFG,0.046060,0.053604,0.073765,0.074233,0.086292,0.101467,0.107637,0.123734,...,0.241723,0.293837,0.412017,0.350371,0.315602,0.299445,South Asia,Low income,Fiscal year end: March 20; reporting period fo...,Afghanistan
2,Angola,AGO,0.097472,0.079038,0.201289,0.192535,0.201003,0.191528,0.246413,0.154912,...,1.232495,1.243406,1.252789,1.330843,1.254617,1.291328,Sub-Saharan Africa,Lower middle income,NaN,Angola
3,Albania,ALB,1.258195,1.374186,1.439956,1.181681,1.111742,1.166099,1.333055,1.363746,...,1.495600,1.578574,1.803715,1.692908,1.749211,1.978763,Europe & Central Asia,Upper middle income,NaN,Albania
4,Andorra,AND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.121652,6.122595,5.867130,5.916597,5.900753,5.832170,Europe & Central Asia,High income,WB-3 code changed from ADO to AND to align wit...,Andorra
6,United Arab Emirates,ARE,0.118758,0.108837,0.163018,0.175223,0.132355,0.146327,0.159950,5.383685,...,21.908556,19.443690,19.076489,19.817684,18.954164,23.302023,Middle East & North Africa,High income,NaN,United Arab Emirates
7,Argentina,ARG,2.367473,2.442616,2.522392,2.316356,2.538379,2.641714,2.792654,2.858162,...,4.410890,4.558500,4.600291,4.569384,4.462904,4.746797,Latin America & Caribbean,Upper middle income,National Institute of Statistics and Census re...,Argentina
8,Armenia,ARM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.509412,1.465622,1.710071,1.976060,1.899712,1.902759,Europe & Central Asia,Lower middle income,NaN,Armenia
10,Antigua and Barbuda,ATG,0.662643,0.849084,1.796794,1.446821,1.540135,2.473585,5.613448,8.912847,...,5.446757,5.539568,5.363407,5.418447,5.360453,5.377649,Latin America & Caribbean,High income,NaN,Antigua and Barbuda
11,Australia,AUS,8.582937,8.641569,8.835688,9.226440,9.759073,10.622321,10.328092,10.955625,...,18.200182,17.740845,17.538878,17.076819,16.103336,15.398600,East Asia & Pacific,High income,Fiscal year end: June 30; reporting period for...,Australia


想要导出为 csv 文件?

In [28]:
mydf.to_csv("resources/data/co2emissions_cleaned.csv", index = False) #See Recent Data for exported csv

<hr></hr>
<div class="alert alert-success alertsuccess" style="margin-top: 0px">
<h4> [tip] 数据清洗的规则来源于对业务的理解 </h4>
<p></p>
从上面的清洗过程我们可以看到，无论是如何区分国家和地区，还是剔除不包含数据的国家，都需要明确的清洗规则，而清洗规则是依靠我们对业务的理解而确定的，单纯从数据本身出发，是无法制定出这样的规则的。
<li>这再次说明，数据分析是业务驱动的，并且需要业务模型的支持。</li>
<li>计算机能够帮助我们的，是依靠强大的计算能力来加速数据分析的过程。</li>
<p></p>
</div>
<hr></hr>